## Import and setup

In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [3]:
import torch
import numpy as np
import pandas as pd
import torch.optim as optim
from utils.preprocessing import *
from models.gsr_model import GSRNet
from utils.gsr_train import train, test
from models.ops import GraphUnet


ModuleNotFoundError: No module named 'layers'

## Define model and hyperparameters

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define model arguments
class Args:
    epochs = 200
    lr = 0.0001
    splits = 5
    lmbda = 16
    lr_dim = 160
    hr_dim = 268  # Adapted for dataset
    hidden_dim = 268
    padding = 0  # No padding

args = Args()

# Load dataset
subjects_adj, subjects_labels, test_adj = data()

# Cross-validation split
from sklearn.model_selection import KFold
cv = KFold(n_splits=args.splits, shuffle=True, random_state=42)

ks = [0.9, 0.7, 0.6, 0.5]  # Pooling rates
model = GSRNet(ks, args).to(device)
optimizer = optim.Adam(model.parameters(), lr=args.lr)


## Train and save model

In [4]:
for train_index, test_index in cv.split(subjects_adj):
    train_adj, val_adj = subjects_adj[train_index], subjects_adj[test_index]
    train_labels, val_labels = subjects_labels[train_index], subjects_labels[test_index]

    train(model, optimizer, train_adj, train_labels, args)
    test(model, val_adj, val_labels, args)

# Save trained model
torch.save(model.state_dict(), "gsr_net_trained.pth")
print("Model saved!")


Epoch: 1, Loss: 1.5674083564514505, Error (MAE): 0.23988297592876548
Epoch: 2, Loss: 1.244831723378117, Error (MAE): 0.20391422835059633
Epoch: 3, Loss: 1.0981995736745964, Error (MAE): 0.19760659583529136
Epoch: 4, Loss: 1.0179410043515658, Error (MAE): 0.1955843179073549
Epoch: 5, Loss: 0.9718425704124278, Error (MAE): 0.19449561285345177
Epoch: 6, Loss: 0.9429309901438261, Error (MAE): 0.19360809252226263
Epoch: 7, Loss: 0.9229554870074853, Error (MAE): 0.1928677919663881
Epoch: 8, Loss: 0.9082591242359993, Error (MAE): 0.19220534919347979
Epoch: 9, Loss: 0.895342242897005, Error (MAE): 0.1916287592927316
Epoch: 10, Loss: 0.883061923478779, Error (MAE): 0.19112721679354072
Epoch: 11, Loss: 0.8711485741730023, Error (MAE): 0.19066869540322096
Epoch: 12, Loss: 0.8596598362564144, Error (MAE): 0.19026086641881698
Epoch: 13, Loss: 0.8478992025655016, Error (MAE): 0.18990123395184824
Epoch: 14, Loss: 0.8366044248853411, Error (MAE): 0.18957540628157163
Epoch: 15, Loss: 0.8254556445250834

In [22]:
# Load trained model
model.load_state_dict(torch.load("gsr_net_trained.pth"))
model.eval()  # Ensure evaluation mode

# Generate predictions
preds_list = []
for lr in test_adj:
    lr = torch.from_numpy(lr).float().to(device)
    with torch.no_grad():  # Disable gradient computation
        preds, _, _, _ = model(lr)  # Get predictions
    preds_list.append(preds.detach().cpu().numpy())  # Convert to NumPy

# Extract only upper triangular part (excluding diagonal)
def upper_triangular_flatten(matrix):
    return matrix[np.triu_indices(268, k=1)]  # k=1 excludes diagonal

# Apply upper triangular flattening to all predictions
preds_list = [upper_triangular_flatten(pred) for pred in preds_list]

# Create DataFrame
submission_df = pd.DataFrame({
    "ID": np.arange(1, len(np.concatenate(preds_list)) + 1),  # Sequential IDs
    "Predicted": np.concatenate(preds_list)  # Flattened predictions
})

# Save CSV
submission_df.to_csv("submission.csv", index=False)
print("Submission file created!")


Submission file created!


## Submit to kaggle

In [21]:
!kaggle competitions submit -c dgl-2025-brain-graph-super-resolution-challenge -f submission.csv -m "GSR-Net submission"

100%|██████████████████████████████████████| 71.2M/71.2M [00:02<00:00, 29.5MB/s]
Successfully submitted to DGL 2025: Brain Graph Super-Resolution Challenge